In [11]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from Google_config import g_key

In [9]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Humidity,Cloudiness,Wind Speed,Date
0,0,Provideniya,RU,44.26,64.3833,-173.3000,93,100,9.57,2022-09-18 01:56:27
1,1,Punta Arenas,CL,39.31,-53.1500,-70.9167,70,20,23.02,2022-09-18 01:56:27
2,2,Khatanga,RU,29.77,71.9667,102.5000,76,21,4.36,2022-09-18 01:56:28
3,3,Longyearbyen,SJ,30.04,78.2186,15.6401,80,75,11.50,2022-09-18 01:56:28
4,4,Jumla,NP,60.87,29.2747,82.1838,91,100,1.86,2022-09-18 01:56:28


In [10]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Max Temp      float64
Lat           float64
Lng           float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Date           object
dtype: object

In [5]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [38]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))
# Assign the figure variable.
fig = gmaps.figure(center=(33.0, -105.0), zoom_level=3.5)
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp])
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(33.0, -105.0), zoom_level=3.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [39]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100


In [40]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Lat,Lng,Humidity,Cloudiness,Wind Speed,Date
12,12,Kapaa,US,83.12,22.0752,-159.3190,73,81,5.01,2022-09-18 01:56:31
14,14,Hambantota,LK,77.34,6.1241,81.1185,100,0,6.91,2022-09-18 01:56:32
22,22,Vaini,TO,75.36,-21.2000,-175.2000,83,75,13.80,2022-09-18 01:56:35
24,24,Hilo,US,86.29,19.7297,-155.0900,82,6,4.00,2022-09-18 01:56:37
32,32,Mahibadhoo,MV,80.87,3.7833,72.9667,71,66,4.47,2022-09-18 01:56:40
34,34,Naze,JP,76.12,28.3667,129.4833,85,100,38.86,2022-09-18 01:56:40
36,36,Avarua,CK,80.65,-21.2078,-159.7750,74,3,13.80,2022-09-18 01:56:41
38,38,Tabou,CI,75.90,4.4230,-7.3528,84,88,9.22,2022-09-18 01:56:42
42,42,Ewa Beach,US,84.43,21.3156,-158.0072,64,37,13.11,2022-09-18 01:56:44
44,44,Lorengau,PG,85.95,-2.0226,147.2712,62,11,10.98,2022-09-18 01:57:44


In [41]:
preferred_cities_df.count()

City_ID       182
City          182
Country       182
Max Temp      182
Lat           182
Lng           182
Humidity      182
Cloudiness    182
Wind Speed    182
Date          182
dtype: int64

In [42]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Kapaa,US,83.12,22.0752,-159.3190,
14,Hambantota,LK,77.34,6.1241,81.1185,
22,Vaini,TO,75.36,-21.2000,-175.2000,
24,Hilo,US,86.29,19.7297,-155.0900,
32,Mahibadhoo,MV,80.87,3.7833,72.9667,
34,Naze,JP,76.12,28.3667,129.4833,
36,Avarua,CK,80.65,-21.2078,-159.7750,
38,Tabou,CI,75.90,4.4230,-7.3528,
42,Ewa Beach,US,84.43,21.3156,-158.0072,
44,Lorengau,PG,85.95,-2.0226,147.2712,


In [66]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [67]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [68]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
12,Kapaa,US,83.12,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Hambantota,LK,77.34,6.1241,81.1185,Bungalow 63
22,Vaini,TO,75.36,-21.2000,-175.2000,Keleti Beach Resort
24,Hilo,US,86.29,19.7297,-155.0900,Hilo Hawaiian Hotel
32,Mahibadhoo,MV,80.87,3.7833,72.9667,Omadhoo Inn
34,Naze,JP,76.12,28.3667,129.4833,Shishime Hotel
36,Avarua,CK,80.65,-21.2078,-159.7750,Paradise Inn
38,Tabou,CI,75.90,4.4230,-7.3528,hotêl doufoulougou
42,Ewa Beach,US,84.43,21.3156,-158.0072,Base Lodging
44,Lorengau,PG,85.95,-2.0226,147.2712,Lorengau Harbourside Hotel


In [71]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [72]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [73]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [74]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))